In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils')
from Preprocessing import Preprocessor
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

preprocessor = Preprocessor()

## Load and preprocess Mmx stations data

In [2]:
%%time
# getting file tree in directory "data_csv", which contains raw unfiltered data
station_list = [113, 114, 612] 
                #115, 116, 117, 118, 119, 1809, 1810, 1811, 1812, 1813, 1814, 1815]
mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/Raw_extended/'

# reading loaded csv files from data_csv directory, output is a list (length=number of stations) of raw df
raw_list = [pd.read_csv(mypath + str(station_id) + '_raw.csv', parse_dates = ['date_time']) 
              for station_id in station_list]
raw = pd.concat(raw_list)
raw = raw.reset_index(drop=True)

CPU times: user 2.16 s, sys: 268 ms, total: 2.43 s
Wall time: 2.43 s


In [3]:
%%time
#features_to_select = ['t_air', 't_road', 't_underroad', 'pressure', 'dampness', 'cloudiness', 'precip_code', 'wind_dir']
#raw_data = preprocessor.SelectFeatures(raw, features_to_select)
raw_data = preprocessor.SelectFeatures(raw)
raw_data = preprocessor.PivotTable(raw_data)
mmx_data = preprocessor.ConvertData(raw_data, from_format="Raw", to_format="Mmx")
mmx_data = preprocessor.AddUTC(mmx_data)
mmx_patterns = preprocessor.CreatePatternList(mmx_data)
mmx_interpolated = preprocessor.InterpolatePatterns(mmx_patterns)

CPU times: user 3.1 s, sys: 408 ms, total: 3.51 s
Wall time: 3.5 s


## Load and preprocess RP5 stations data

In [4]:
%%time
#station_list = [28719]
station_list = [22831, 22925, 22867, 28696, 31318, 31917, 28719]
mypath = '/mnt/HARD/MinMax94/data/data_all/CSV/RP5/'

rp5_list = []
for station_id in station_list:
    df = pd.read_csv(mypath + str(station_id) + '.csv', sep=';', skiprows=6, index_col=False)
    date_time_col = [col for col in df.columns if col.startswith('Местное время')][0]
    df = df.rename(columns={date_time_col: 'Местное время'})
    df['station_id'] = station_id
    rp5_list.append(df)
rp5 = pd.concat(rp5_list).reset_index(drop=True)
#rp5_data

CPU times: user 487 ms, sys: 0 ns, total: 487 ms
Wall time: 486 ms


In [5]:
rp5_data = preprocessor.ConvertData(rp5, from_format="RP5", to_format="Mmx")
del rp5_data['data_p_weather'], rp5_data['data_precip_interval']
rp5_data = preprocessor.AddUTC(rp5_data, '/mnt/HARD/MinMax94/data/data_all/CSV/stations_rp5_def.csv')
rp5_interpolated = preprocessor.InterpolatePatterns(rp5_data)

## Convert Mmx data to METRO format

In [7]:
from Preprocessing import vincenty_dist

In [8]:
global_forecast = preprocessor.ConvertData(mmx_interpolated, from_format='Mmx', to_format='Metro')

In [9]:
station_id = 113

stations_mmx = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_mm94_def.csv', index_col=0)
stations_rp5 = pd.read_csv('/mnt/HARD/MinMax94/data/data_all/CSV/stations_rp5_def.csv', index_col=0)

station_data = stations_mmx[stations_mmx.index==station_id]
lat, lon = station_data[['latitude', 'longitude']].values[0]
timezone = station_data['timezone'].values[0]

In [10]:
from Preprocessing import vincenty_dist
nearest_wmo_id = stations_rp5.loc[stations_rp5.apply(lambda x: vincenty_dist((lat, lon), (x['latitude'], x['longitude'])), 
                                        axis=1).idxmin()].name
nearest_wmo_id

28719

In [24]:
global_forecast = preprocessor.ConvertData(mmx_interpolated, from_format='Mmx', to_format='Metro')
global_forecast = global_forecast.set_index('date_time_utc')

In [21]:
current_date_time = pd.Timestamp(2015, 2, 10, 10, 0)
predict_delta = pd.Timedelta(48, unit='h')

In [ ]:
global_forecast_point = global_forecast